In [1]:
import json
import os
import random
from typing import List, Dict
import uuid

from openai import OpenAI, AsyncOpenAI
from pydantic import BaseModel, Extra, Field
import tiktoken

from config import settings, app_settings

from src.simschat.models import (
    CharacterSpecification,
    SceneState,
    CharacterRelationState,
    CharacterState,
    create_dynamic_enum
)

In [2]:
openai_client = OpenAI(api_key=settings.openai_api_key)
tokenizer = tiktoken.encoding_for_model("gpt-4o")

In [3]:
class Character(BaseModel):
    uid: str
    spec: CharacterSpecification
    state: CharacterState

# 0. Load Character & State

In [4]:
# Load Scene
scene_uid = "2f2462c1-bdef-4089-b206-c47decd841f3"

with open(f"simschat/scene/{scene_uid}.json", "r") as f:
    scene_dict = json.load(f)

In [5]:
scene_description = scene_dict["scene"]["revised_trope"]
scene_state = SceneState.model_validate(scene_dict["scene"]["scene"])

In [6]:
character_states = {
    scene_dict["character_states"]["character1_uid"]: CharacterState.model_validate(scene_dict["character_states"]["character1"]),
    scene_dict["character_states"]["character2_uid"]: CharacterState.model_validate(scene_dict["character_states"]["character2"]),
}

In [7]:
# Load Character
with open("simschat/character_collection.json", "r") as f:
    character_collection = json.load(f)

character_ids = list(character_collection["source"].keys())

character1_id="35f0c56f-263d-42df-846c-e1833d8ca0ab"
character2_id="00d66087-9b3b-46da-bd74-bf45cbe81d3c"

print(character1_id, character_collection["source"][character1_id])
print(character2_id, character_collection["source"][character2_id])

35f0c56f-263d-42df-846c-e1833d8ca0ab wiki_Zephyr Orion.txt
00d66087-9b3b-46da-bd74-bf45cbe81d3c wiki_Vivienne LaRoux.txt


In [8]:
# Load Characters
with open(f"simschat/characters/{character1_id}.json", "r") as f:
    character1_spec = CharacterSpecification(**json.load(f))
    
with open(f"simschat/characters/{character2_id}.json", "r") as f:
    character2_spec = CharacterSpecification(**json.load(f))

In [9]:
character1 = Character(
    uid=character1_id,
    spec=character1_spec,
    state=character_states[character1_id]
)
character2 = Character(
    uid=character2_id,
    spec=character2_spec,
    state=character_states[character2_id]
)

In [10]:
print(character1.model_dump_json(indent=4))

{
    "uid": "35f0c56f-263d-42df-846c-e1833d8ca0ab",
    "spec": {
        "name": "Zephyr Orion",
        "gender": "male",
        "age": 28,
        "dialogue_tone": "playful, jovial, and engaging, with a witty humor and warmth that makes everyone feel at ease. Known for storytelling with captivating tales of space adventures.",
        "career": "Astronaut",
        "personality_traits": [
            {
                "trait": "Goofball",
                "description": "Enjoys joking and making others laugh, bringing a playful spirit to social situations."
            },
            {
                "trait": "Materialistic",
                "description": "Loves acquiring new possessions and often leans towards bragging about them."
            },
            {
                "trait": "Outgoing",
                "description": "Flourishes in social situations and enjoys being around people."
            },
            {
                "trait": "Gloomy",
                "descrip

# 1. Action Prediction
* utilize the 'Social interaction' from sims
    * https://sims.fandom.com/wiki/Social_interaction

## Sims interactions
* Friendly Interactions
* Romantic Interactions
* Mean Interactions
* Neutral Interactions

In [11]:
import random
with open("sims_interactions.json", "r") as f:
    full_sims_interactions = json.load(f)

# sample    
n = 10
sims_interactions = {k: random.sample(v, min(len(v), n)) for k,v in full_sims_interactions.items()}
sims_interactions.keys()

dict_keys(['Friendly', 'Ask', 'Romantic', 'Mean', 'Neutral'])

In [27]:
## Make enum of actions
import itertools
possible_actions_model = create_dynamic_enum(
    "PossibleCharacterActions",
    itertools.chain(*[
        [f"{k}-{a['action']}" for a in v]
        for k,v in sims_interactions.items()
    ])
)

In [28]:
class CharacterAction(BaseModel):
    character: str
    think: str
    action_type: possible_actions_model
    description: str
    dialogue: str
    targets: List[str]
    class Config:
        extra = Extra.forbid

class ActionsResult(BaseModel):
    actions: List[CharacterAction]
    class Config:
        extra = Extra.forbid

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_40016/2445234414.py:9: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid
/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_40016/2445234414.py:14: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid


In [29]:
ActionsResult.model_json_schema()

{'$defs': {'CharacterAction': {'additionalProperties': False,
   'properties': {'character': {'title': 'Character', 'type': 'string'},
    'think': {'title': 'Think', 'type': 'string'},
    'action_type': {'$ref': '#/$defs/PossibleCharacterActions'},
    'description': {'title': 'Description', 'type': 'string'},
    'dialogue': {'title': 'Dialogue', 'type': 'string'},
    'targets': {'items': {'type': 'string'},
     'title': 'Targets',
     'type': 'array'}},
   'required': ['character',
    'think',
    'action_type',
    'description',
    'dialogue',
    'targets'],
   'title': 'CharacterAction',
   'type': 'object'},
  'PossibleCharacterActions': {'enum': ['Friendly-Ask if Sim Slept Well',
    'Friendly-Compliment',
    'Friendly-Deep Conversation',
    'Friendly-Cheer Up',
    'Friendly-Hug',
    'Friendly-Announce Pregnancy',
    'Friendly-Talk about ...',
    'Friendly-Request Feel Tummy',
    'Friendly-Get to Know',
    'Friendly-Give Backrub',
    'Ask-About Interests',
    '

In [30]:
ACTION_PREDICTION_INSTRUCTION = '''Given the following informations about a story scene, write a sequence of actions that the characters will perform in the scene.

# Given Informations
* Characters: 2 characters involved in the story. (identified through their uid)
* Scene Trope: the story trope that the given scene follows
* Scene Setting: location, setting of the scene

## Characters
[Character1]
{character1}

[Character2]
{character2}

## Scene Trope
{trope}

## Scene Setting
{scene}

# Action Prediction
## Definition of 'Action'
a character performs an 'action' through the following steps
* 'character': select the character that will perform this action (by uid)
* 'think': consider what the character should say and what goals the character wants to achieve. write the thinking process here
* 'action_type': type of the action this character will actually perform
* 'description': description of the action this character will perform
    * the action must be detailed description of a single-verb action.
    * if there are dialogues, write the actual dialogues
    * if there are any background, items, characters involved, include the details about them
    * action must be maximum 2 sentences
    * don't make the actions obvious
* 'dialogue': the dialogue this character will perform (leave it empty if there are no dialogues)
* 'targets': List of characters (by uid) that will be affected. leave it empty if it there are no targets

### Action Type
type of actions that the character can perform are defined as the following
* actions are grouped by their 'Interaction' type (ex. Fiendly, Mean, ..)
* select the action in the form of "{{interaction}}-{{action}}" from the following dictionary (ex. Friendly-Admire)
{actions}

Return a list of actions in the following json format (maximum of 3)
{{
    "actions": [
        {{
            "character": str,
            "think": str
            "action": str,
            "targets": List[str]
        }},
        ...
    ]
}}'''

In [31]:
action_pred_instruction = ACTION_PREDICTION_INSTRUCTION.format(
    character1=character1.model_dump_json(),
    character2=character2.model_dump_json(),
    trope=scene_description,
    scene=scene_state.model_dump_json(),
    actions=json.dumps(sims_interactions)
)

In [32]:
messages = [
    {"role": "user", "content": action_pred_instruction}
]
# decode_params = {"temperature": 0.95}
decode_params = {"reasoning_effort": "low"}

response = openai_client.beta.chat.completions.parse(
    # model="gpt-4o",
    model="o3-mini",
    messages=messages,
    response_format=ActionsResult,
    **decode_params,
)
predicted_actions = response.choices[0].message.parsed.actions

In [37]:
response.usage.model_dump()

{'completion_tokens': 943,
 'prompt_tokens': 3123,
 'total_tokens': 4066,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 576,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

In [33]:
print(len(predicted_actions))

2


In [34]:
for action in predicted_actions:
    print(action.model_dump_json(indent=4))
    print('-'*30)

{
    "character": "35f0c56f-263d-42df-846c-e1833d8ca0ab",
    "think": "Zephyr aims to melt Vivienne's icy demeanor with his playful charm and genuine curiosity about her sophisticated world.",
    "action_type": "Friendly-Get to Know",
    "description": "Zephyr strides over with a bright smile and an easy laugh, engaging Vivienne with warm, inquisitive dialogue while subtly referencing the art and fashion around them.",
    "dialogue": "So, Vivienne, what brings a style icon like you to a night of space dreams and couture marvels?",
    "targets": [
        "00d66087-9b3b-46da-bd74-bf45cbe81d3c"
    ]
}
------------------------------
{
    "character": "00d66087-9b3b-46da-bd74-bf45cbe81d3c",
    "think": "Vivienne is initially dismissive but finds herself unexpectedly drawn in by Zephyr's charisma, leading her to deliver a sharp remark that masks a hint of intrigue.",
    "action_type": "Mean-Mock Outfit",
    "description": "Vivienne raises an eyebrow, her tone coolly sardonic as s